## Libraries

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stat

from math import sqrt
from mlgear.utils import show, display_columns
from surveyweights import normalize_weights


def margin_of_error(n=None, sd=None, p=None, type='proportion', interval_size=0.95):
    z_lookup = {0.8: 1.28, 0.85: 1.44, 0.9: 1.65, 0.95: 1.96, 0.99: 2.58}
    if interval_size not in z_lookup.keys():
        raise ValueError('{} not a valid `interval_size` - must be {}'.format(interval_size,
                                                                              ', '.join(list(z_lookup.keys()))))
    if type == 'proportion':
        se = sqrt(p * (1 - p)) / sqrt(n)
    elif type == 'continuous':
        se = sd / sqrt(n)
    else:
        raise ValueError('{} not a valid `type` - must be proportion or continuous')
    
    z = z_lookup[interval_size]
    return se * z


def print_pct(pct, digits=0):
    pct = pct * 100
    pct = np.round(pct, digits)
    if pct >= 100:
        if digits == 0:
            val = '>99.0%'
        else:
            val = '>99.'
            for d in range(digits - 1):
                val += '9'
            val += '9%'
    elif pct <= 0:
        if digits == 0:
            val = '<0.1%'
        else:
            val = '<0.'
            for d in range(digits - 1):
                val += '0'
            val += '1%'
    else:
        val = '{}%'.format(pct)
    return val


def calc_result(dem_vote, rep_vote, n, interval=0.8):
    GENERAL_POLLING_ERROR = 2.5
    TIME_SHIFT_ERROR = 0.5
    N_SIMS = 100000
    
    dem_moe = margin_of_error(n=n, p=dem_vote/100, interval_size=interval)
    rep_moe = margin_of_error(n=n, p=rep_vote/100, interval_size=interval)
    undecided = 100 - dem_vote - rep_vote

    dem_mean = dem_vote + undecided * 0.25
    dem_raw_moe = dem_moe * 100
    dem_allocate_undecided = undecided * 0.4
    dem_margin = dem_raw_moe + dem_allocate_undecided + GENERAL_POLLING_ERROR + TIME_SHIFT_ERROR
    
    rep_mean = rep_vote + undecided * 0.25
    rep_raw_moe = rep_moe * 100
    rep_allocate_undecided = undecided * 0.4
    rep_margin = rep_raw_moe + rep_allocate_undecided + GENERAL_POLLING_ERROR + TIME_SHIFT_ERROR
    
    cdf_value = 0.5 + 0.5 * interval
    normed_sigma = stat.norm.ppf(cdf_value)
    
    dem_sigma = dem_margin / 100 / normed_sigma
    dem_sims = np.random.normal(dem_mean / 100, dem_sigma, N_SIMS)
    
    rep_sigma = rep_margin / 100 / normed_sigma
    rep_sims = np.random.normal(rep_mean / 100, rep_sigma, N_SIMS)
    
    chance_pass = np.sum([sim[0] > sim[1] for sim in zip(dem_sims, rep_sims)]) / N_SIMS
    
    low, high = np.percentile(dem_sims - rep_sims, [20, 80]) * 100
    
    return {'mean': dem_mean - rep_mean, 'high': high, 'low': low, 'n': n,
            'raw_moe': dem_raw_moe + rep_raw_moe,
            'margin': (dem_margin + rep_margin) / 2,
            'sigma': (dem_sigma + rep_sigma) / 2,
            'chance_pass': chance_pass}


def print_result(mean, high, low, n, raw_moe, margin, sigma, chance_pass):
    mean = np.round(mean, 1)
    first = np.round(high, 1)
    second = np.round(low, 1)
    sigma = np.round(sigma * 100, 1)
    raw_moe = np.round(raw_moe, 1)
    margin = np.round(margin, 1)
    chance_pass = print_pct(chance_pass, 1)
    if second < first:
        _ = first
        first = second
        second = _
    if second > 100:
        second = 100
    if first < -100:
        first = -100
    print(('Result Dems +{} (80% CI: {} to {}) (Weighted N={}) (raw_moe={}pts, margin={}pts, '
           'sigma={}pts) (Dems {} likely to win)').format(mean,
                                                          first,
                                                          second,
                                                          n,
                                                          raw_moe,
                                                          margin,
                                                          sigma,
                                                          chance_pass))
    print('-')

## Load Processed Data

In [2]:
survey = pd.read_csv('responses_processed_national_weighted.csv').fillna('Not presented')

/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,40,41,42,43,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Generic Congressional Ballot

In [3]:
survey['vote_rep'].value_counts(normalize=True)

A Democratic candidate    0.588080
A Republican candidate    0.189743
Not decided               0.104804
Do not intend to vote     0.076830
Another candidate         0.018244
Unsure                    0.014190
I did not vote            0.008109
Name: vote_rep, dtype: float64

In [4]:
options = ['A Democratic candidate', 'A Republican candidate', 'Another candidate', 'Not decided']
survey_ = survey.loc[survey['vote_rep'].isin(options)].copy()
survey_['weight'] = normalize_weights(survey_['weight'])
survey_['rv_weight'] = normalize_weights(survey_['rv_weight'])
survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])
    
print('## NATIONAL UNWEIGHTED ##')
n = len(survey_)
votes = survey_['vote_rep'].value_counts(normalize=True) * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result(**calc_result(dem_vote=votes['A Democratic candidate'],
                           rep_vote=votes['A Republican candidate'],
                           n=n))
    
print('## NATIONAL WEIGHTED ##')
weighted_n = int(np.round(survey_['weight'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_rep'].value_counts(normalize=True) * survey_.groupby('vote_rep')['weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result(**calc_result(dem_vote=votes['A Democratic candidate'],
                           rep_vote=votes['A Republican candidate'],
                           n=weighted_n))

print('## NATIONAL WEIGHTED + RV ##')
rv_weighted_n = int(np.round(survey_['rv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_rep'].value_counts(normalize=True) * survey.groupby('vote_rep')['rv_weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result(**calc_result(dem_vote=votes['A Democratic candidate'],
                           rep_vote=votes['A Republican candidate'],
                           n=rv_weighted_n))

print('## NATIONAL WEIGHTED + LV ##')
lv_weighted_n = int(np.round(survey_['lv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_rep'].value_counts(normalize=True) * survey.groupby('vote_rep')['lv_weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result(**calc_result(dem_vote=votes['A Democratic candidate'],
                           rep_vote=votes['A Republican candidate'],
                           n=lv_weighted_n))

## NATIONAL UNWEIGHTED ##
A Democratic candidate    65.279028
A Republican candidate    21.062106
Another candidate          2.025203
Not decided               11.633663
Name: vote_rep, dtype: float64
Result Dems +44.2 (80% CI: 35.6 to 52.8) (Weighted N=4444) (raw_moe=1.7pts, margin=9.3pts, sigma=7.3pts) (Dems >99.9% likely to win)
-
## NATIONAL WEIGHTED ##
A Democratic candidate    45.595352
A Republican candidate    40.044352
Another candidate          2.453497
Not decided               11.906800
dtype: float64
Result Dems +5.6 (80% CI: -3.9 to 15.0) (Weighted N=1835) (raw_moe=3.0pts, margin=10.2pts, sigma=8.0pts) (Dems 68.8% likely to win)
-
## NATIONAL WEIGHTED + RV ##
A Democratic candidate    47.697772
A Republican candidate    42.003772
Another candidate          2.401967
Not decided                7.896488
dtype: float64
Result Dems +5.7 (80% CI: -2.3 to 13.7) (Weighted N=1790) (raw_moe=3.0pts, margin=8.6pts, sigma=6.7pts) (Dems 72.5% likely to win)
-
## NATIONAL WEIGHTED + LV 